This notebook first determines
1. the number of Eukaryotic GenBank genomes that have gene models (`CDS_from_genomic`) available for download
2. the number of phyla that have gene models

In [5]:
setwd("..")

In [6]:
library(readr)
library(dplyr, warn.conflicts = F)
library(purrr)
library(tidyr)
library(janitor, warn.conflicts = F)
library(taxize)
library(ggplot2)
library(ggforce)

## GenBank genomes with gene models

The genbank `annotation_hashes.txt` file has a columns named "Features hash" and "Proteins name hash."
When the values of either of those columns are "D41D8CD98F00B204E9800998ECF8427E", that indicates the file does not exist.

In [3]:
# don't spam NCBI and get IP blocked; delays the read_tsv function by 6 seconds
slowly_read_tsv <- slowly(read_tsv, rate = rate_delay(6))

In [4]:
# generate urls to download
genbank_url_prefix <- "https://ftp.ncbi.nlm.nih.gov/genomes/genbank/"
genbank_groups <- c("plant", "fungi", "invertebrate", "protozoa", "vertebrate_mammalian", "vertebrate_other")
annotation_hashes_urls <- paste0(genbank_url_prefix, genbank_groups, "/annotation_hashes.txt")
assembly_summaries_urls <- paste0(genbank_url_prefix, genbank_groups, "/assembly_summary.txt")

In [5]:
annotation_hashes <- annotation_hashes_urls %>%
  set_names()  %>%   # set names (interacts with .id below to use the URL as the genbank_group col)
  map_dfr(slowly_read_tsv, show_col_types = F, .id = "genbank_group",) %>%  # read in each TSV from NCBI, pausing for 6 seconds after each run
  clean_names() %>%  # clean column names (captilization, spacing, etc.)
  mutate(genbank_group = gsub(genbank_url_prefix, "", genbank_group),
         genbank_group = gsub("/annotation_hashes.txt", "", genbank_group)) # clean genbank_group to remove URL cruft leaving the group names themselves

In [6]:
does_not_exist_hash <- "D41D8CD98F00B204E9800998ECF8427E"
table(annotation_hashes$features_hash == does_not_exist_hash)
table(annotation_hashes$protein_names_hash == does_not_exist_hash)


FALSE  TRUE 
 7276 27044 


FALSE  TRUE 
 6179 27721 

In [9]:
# what percentage of genomes in GenBank have gene models?
7276/(7276+27044)

[1] 0.2120047

In [10]:
# get metadata, including organism name, for GenBank genomes
assembly_summaries <- assembly_summaries_urls %>%
  set_names() %>% # set names (interacts with .id below to use the URL as the genbank_group col)
  map_dfr(slowly_read_tsv, .id = "genbank_group", skip = 1, show_col_types = F) %>% # read in each TSV from NCBI, pausing for 6 seconds after each run
  mutate(genbank_group = gsub(genbank_url_prefix, "", genbank_group),
         genbank_group = gsub("/assembly_summary.txt", "", genbank_group)) # clean genbank_group to remove URL cruft leaving the group names themselves

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details, e.g.:
  dat <- vroom(...)
  problems(dat)”


In [13]:
genbank_genomes <- left_join(annotation_hashes, assembly_summaries, by = c("number_assembly_accession" = "#assembly_accession", "genbank_group"))
genbank_genomes_cds <- genbank_genomes %>%
  filter(features_hash != does_not_exist_hash)

In [14]:
nrow(genbank_genomes)
nrow(genbank_genomes_cds)

[1] 34320

[1] 7276

# Get taxonomy information from taxonkit

In [22]:
# taxonkit version 0.14.2
system('taxonkit reformat -I 15 -f "{k};{K};{p};{c};{o};{f};{g};{s};{t}" --data-dir sandbox/try_blast -o sandbox/count_phyla_with_gene_models/20230630_genbank_genomes_lineages.tsv inputs/20230630_genbank_genomes.tsv.gz')

# Analyze

In [11]:
lineages <- read_tsv("sandbox/count_phyla_with_gene_models/20230630_genbank_genomes_lineages.tsv", show_col_types = F) %>%
  rename(lineage = `...48`) %>%
  separate(lineage, into = c("superkingdom", "kingdom", "phylum", "class", "order", "family", "genus", "species", "strain"), sep = ";")

New names:
• `` -> `...48`


In [13]:
# how many phyla are there?
length(unique(lineages$phylum))

[1] 56

In [20]:
# how many phyla have at least one genome with gene models?
lineages_gene_models <- lineages %>% 
  filter(features_hash != does_not_exist_hash)
length(unique(lineages_gene_models$phylum))

[1] 45

In [21]:
sessionInfo()

R version 4.3.0 (2023-04-21)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Big Sur ... 10.16

Matrix products: default
BLAS/LAPACK: /Users/taylorreiter/miniconda3/envs/tidyjupyter/lib/libopenblasp-r0.3.23.dylib;  LAPACK version 3.11.0

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

time zone: America/New_York
tzcode source: system (macOS)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] ggforce_0.4.1  ggplot2_3.4.2  taxize_0.9.100 janitor_2.2.0  tidyr_1.3.0   
[6] purrr_1.0.1    dplyr_1.1.2    readr_2.1.4   

loaded via a namespace (and not attached):
 [1] utf8_1.2.3        generics_0.1.3    xml2_1.3.4        stringi_1.7.12   
 [5] lattice_0.21-8    httpcode_0.3.0    hms_1.1.3         digest_0.6.31    
 [9] magrittr_2.0.3    evaluate_0.21     grid_4.3.0        timechange_0.2.0 
[13] pbdZMQ_0.3-9      iterators_1.0.14  fastmap_1.1.1     foreach_1.5.2    
[17] j